In [48]:
import torch
import numpy as np
import pandas as pd 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [49]:
#Checking if pytorch is installed
torch.__version__

'1.7.1+cpu'

In [50]:
#importing
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None,engine='python',encoding='latin-1')
movies.head(10)
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None,engine='python',encoding='latin-1')
users.head(10) #2nd column - gender , 3rd column - age, 4th column - occupation 5th column - zip code
ratings = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')
ratings.head(10) #1st column users 2nd column - movies(movie id) 3rd column -  ratings 4th column - dont care lol. they are timestamps

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [51]:
#preparing test and train set
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t',header=None)
training_set.head(10)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
5,1,7,4,875071561
6,1,8,1,875072484
7,1,9,5,878543541
8,1,11,2,875072262
9,1,13,5,875071805


In [52]:
#converting training set to array
training_set = np.array(training_set, dtype='int')
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [53]:
test_set = pd.read_csv('ml-100k/u1.test',delimiter='\t',header=None)
test_set = np.array(test_set, dtype='int')
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [54]:
#Getting number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
print(nb_users,nb_movies)

943 1682


In [55]:
#Converting data to array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users] #movies what the user watched
        id_ratings = data[:,2][data[:,0] == id_users] #movies rated by the user
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [56]:
len(training_set)

943

In [57]:
#Converting to torch tensors
#tensors - array contain elements of a single datatype
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [58]:
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [59]:
#Converting the ratings into binary ratings 1 (Liked) or 0 (non liked)
training_set[training_set == 0] = -1 #movie isn't rated
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1 #movie isn't rated
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

#Creating our NN
class RBM():
    def __init__(self,nv,nh): #nv - visible nodes nh - hidden nodes
        self.W = torch.randn(nh,nv) #initialises the weights
        self.a = torch.randn(1,nh) #creating a 2D tensor, nh corresponds to the bias
        self.b = torch.randn(1,nv) #creating the bias for the visible nodes
    
    def sample_h(self,x):
        #x is the visible node here
        wx = torch.mm(x,self.W.t()) #product of two torch tensors
        activation = wx + self.a.expand_as(wx)#sigmoid activation function
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self,y):
        
        wy = torch.mm(y,self.W) #product of two torch tensors
        activation = wy + self.b.expand_as(wy)#sigmoid activation function. bias of visible nodes -> b
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self,v0,vk,ph0, phk): #vo-> input vector (ratings of all movies by a single user),vk-> visible nodes after k iterations, ph0 - vectorof probabilities, phk- Probability of hidden nodes
        #implementing the constrative divergence algorithm
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(),phk)
        self.b += torch.sum((v0-vk),0)
        self.a += torch.sum((ph0-phk),0)

        

In [60]:
class RBM():
    
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
  def predict(self,x):
    _,h = self.sample_h(x)
    _,v = self.sample_v(h)
    return v

In [61]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv,nh)

In [62]:
#Training the RBM
#no of epochs
nb_epoch = 10 

for epoch in range(1, nb_epoch + 1):
    #introduce loss variable
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user: id_user + batch_size]#input batch of ratings
        v0 = training_set[id_user: id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        
        for k in range(10):
            _,hk = rbm.sample_h(vk) #hidden obtained at kth step of divergence
            _,vk = rbm.sample_v(hk) #sampled visible nodes
            vk[v0 < 0]= v0[v0 < 0] #visible nodes
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >=0] - vk[v0>=0]))
        s += 1.
    print('Epoch: {0}. Loss {1}'.format(str(epoch), str(train_loss/s),))

Epoch: 1. Loss tensor(0.3438)
Epoch: 2. Loss tensor(0.2534)
Epoch: 3. Loss tensor(0.2488)
Epoch: 4. Loss tensor(0.2501)
Epoch: 5. Loss tensor(0.2448)
Epoch: 6. Loss tensor(0.2503)
Epoch: 7. Loss tensor(0.2462)
Epoch: 8. Loss tensor(0.2472)
Epoch: 9. Loss tensor(0.2488)
Epoch: 10. Loss tensor(0.2495)


In [63]:
#Testing
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2438)


In [70]:
y_pred = rbm.predict(test_set[34:35])
print(len(y_pred[y_pred > 0]))

1108


In [68]:
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller
